# Adding Agents with RAGS

## Introduction

In this lab, you'll harness the power of Retrieval-Augmented Generation (RAG) and combine it with your custom Google_map tool from Lab2. Your challenge is to integrate these two components: use the Google_map tool to answer location-based queries, and leverage RAG (from Lab3) to provide context-rich answers from your company policy document. By merging tool use and RAG, you'll build an advanced agent capable of both retrieving document knowledge and interacting with external tools for dynamic, real-world tasks.

<table><tr>
<td><img src="../../images/robo1.png" alt="Robo1 - RAG Expert" width="120" /></td>
<td style="vertical-align:top; padding-left:20px;">
<b>Robo1 says:</b><br>
<i>Can you do WFH today? Ask me.. I know the answer</i>
</td>
</tr></table>


## Challenge: Create a RAG from Company Policy Document

Your task is to build a Retrieval-Augmented Generation (RAG) system using the company policy document. 

# Import Libraries and Setup
Let's import the necessary libraries and set up our environment for RAG-powered resume Q&A.

In [33]:
import googlemaps
from langchain.tools import tool
from langchain.agents import create_tool_calling_agent, AgentExecutor
import logging
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent, AgentType


### Load OpenAPI Key from .env File

To securely access the OpenAPI key, we will load it from a `.env` file using environment variable management tools. This ensures sensitive credentials are not exposed in the notebook.

In [34]:
# Load environment variables from .env file
load_dotenv()

# Get OpenAI API key from environment
openai_api_key = os.getenv("OPENAI_API_KEY")

### Create a Retreiver
- load your company policy PDF 
- split it into smaller chunks (embeddings)
- create a vector store from embeddings
- create a retriever

In [67]:
pdf_path = "company_policy.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()

# Split resume into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 8})


### Create a tool using Google Map API

In [68]:
google_map_key= os.getenv("GOOGLE_MAPS_API_KEY")
gmaps = googlemaps.Client(key=google_map_key)

@tool
def get_travel_time(origin: str, destination: str, mode: str = "driving") -> str:
    """Get travel time between two places using Google Maps (modes: driving, walking, bicycling, transit)."""
    directions = gmaps.directions(
        origin=origin,
        destination=destination,
        mode=mode,
        departure_time="now"  # Use live traffic data
    )
    travel_time = directions[0]["legs"][0].get("duration_in_traffic", directions[0]["legs"][0]["duration"])
    travel_time_text = travel_time["text"]
    return f"Estimated travel time: {travel_time_text}"

### Intialize LLM with OpenAPI Key
- Intialize LLM
- Create RetreievalQA

In [69]:
from langchain.tools import Tool

llm = ChatOpenAI(model="gpt-4", temperature=0, api_key=openai_api_key)
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

@tool
def retriever_search(query: str) -> str:
    """Get company policy information based on the query."""
    return qa_chain.run(query)

### Create Appropriate Prompt Template

In [ ]:
prompt_template = """
You are a Commuter Assistant Agent for TechNova Solutions Pvt. Ltd.

Your responsibilities:
1. If the user asks directly about company policy (e.g., "What is my WFH policy?", "What is the leave policy?"):
   - Use `retriever_search` to fetch the exact company policy text.
   - Return the answer using the retrieved content.

2. If the user asks a conditional question (e.g., "Can I WFH if I am travelling from Noida to Gurgaon?"):
   - First, use `retriever_search` to fetch the relevant company policy.
   - Then, use `get_travel_time` to calculate commute time.
   - APPLY the rule from the retrieved policy to the travel time result.
   - Give a clear YES/NO decision with reasoning.

3. If the user only asks about travel or traffic (without policy), use `get_travel_time`.

4. If the answer cannot be found in the retrieved company document, reply only:
   "I don't know based on the company document."

Always be clear, concise, and professional.

Question: {input}
{agent_scratchpad}
Answer as if you are the Assistant Agent:
"""

### Create Agent
- Create Agent
- Create Agent Executor

In [ ]:
agent = create_tool_calling_agent(llm, [get_travel_time], PromptTemplate.from_template(prompt_template))
executor = AgentExecutor(agent=agent, tools=[get_travel_time,retriever_search])

In [ ]:
from langchain.agents import AgentExecutor, create_openai_functions_agent

agent = create_openai_functions_agent(
    llm=llm,
    tools=[get_travel_time,retriever_search],
    prompt=PromptTemplate.from_template(prompt_template),
)

executor = AgentExecutor(agent=agent, tools=[get_travel_time,retriever_search], 
                          handle_parsing_errors=True,
                          return_intermediate_steps=True,
                          verbose=True)

ValueError: ZeroShotAgent does not support multi-input tool get_travel_time.

### Chat with Your Company Policy Agent
Ask questions about your company policy.

In [74]:
import asyncio

async def async_chat_loop(executor):
    print("Company Policy RAG Chatbot (Async Mode). Type your question (or 'exit' to quit):")
    while True:
        question = input("You: ")
        if question.lower() == "exit":
            print("Goodbye!")
            break
        print(f"[DEBUG] Agent received question: {question}")
        response = await asyncio.to_thread(executor.invoke, {"input": question})
        print("Bot:", response.get("output", response))

# To run the chat loop, uncomment the line below:
await async_chat_loop(executor)

Company Policy RAG Chatbot (Async Mode). Type your question (or 'exit' to quit):
[DEBUG] Agent received question: can i do wfh? i am travelling from noida to gurgaon?


> Entering new AgentExecutor chain...

Invoking: `retriever_search` with `{'query': 'work from home policy'}`


The Work-from-Home (WFH) policy at TechNova Solutions Pvt. Ltd. allows employees to WFH up to 2 days per week with manager approval. WFH may also be mandated during severe weather, transport strikes, or when the Air Quality Index (AQI) is greater than 200. If travel time is more than 1 hour, employees may be allowed to do WFH. Remote attendance must be logged through the HR portal. When accessing internal systems remotely, employees must use VPN.Based on the Work-from-Home (WFH) policy at TechNova Solutions Pvt. Ltd., you are allowed to WFH up to 2 days per week with manager approval. If your travel time is more than 1 hour, you may be allowed to do WFH. Let's check your travel time from Noida to Gurgaon. Plea

---

<table><tr>
<td><img src="../../images/robo1.png" alt="Robo1 - RAG Expert" width="120" /></td>
<td style="vertical-align:top; padding-left:20px;">
<b>Robo1 says:</b><br>
<i>"Congratulations! You just built a RAG-powered chatbot."</i><br>
<i>With RAG, your AI can find answers in documents faster than ever. 📄🤖</i>
</td>
</tr></table>

*Thanks for completing the LangChain RAG Lab!*